Be sure to activate the "biopython" conda environment.

In [10]:
# !jupyter-nbextension enable nglview --py --sys-prefix

In [6]:
from Bio.PDB import *
import nglview as nv
import ipywidgets

import warnings
warnings.filterwarnings('ignore')

In [2]:
parser = PDBParser()

In [7]:
structure = parser.get_structure("PHA-L", "Data/1FAT.pdb")

In [8]:
view = nv.show_biopython(structure)
view

NGLWidget()